In [3]:
import importlib
from tools import network_tools as network_tools
from tools import firewall_tools as firewall_tools
importlib.reload(network_tools)
importlib.reload(firewall_tools)

<module 'tools.firewall_tools' from '/home/federico/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/tools/firewall_tools.py'>

In [4]:
alerts = await network_tools.get_alerts(time_window=5)
len(alerts.get('security_events', []).get('alerts'))

0

In [10]:
alerts = await network_tools.get_alerts(time_window=2)
alerts

{'security_events': {'alerts': [{'category': '',
    'dest_ip': '192.168.200.1',
    'dest_port': 36030,
    'flow_id': 2139992503996592,
    'payload': 'HTTP/1.1 200 OK\r\nServer: Werkzeug/3.1.3 Python/3.10.12\r\nDate: Tue, 14 Oct 2025 11:26:31 GMT\r\nContent-Type: application/json\r\nContent-Length: 20400\r\nConnection: close\r\n\r\n{"alerts":[{"category":"Potentially Bad Traffic","dest_ip":"172.20.0.3","dest_port":3306,"flow_id":1151622773388408,"payload":null,"proto":"TCP","severity":2,"signature":"ET SCAN Suspicious inbound to mySQL port 3306","src_ip":"192.168.100.2","src_port":60995,"timestamp":"2025-10-14T13:25:32.399205+0200"},{"category":"Attempted I',
    'proto': 'TCP',
    'severity': 3,
    'signature': 'Root-Level File Access)',
    'src_ip': '192.168.200.2',
    'src_port': 7000,
    'timestamp': '2025-10-14T13:26:31.828074+0200'},
   {'category': 'Attempted Information Leak',
    'dest_ip': '192.168.200.1',
    'dest_port': 36030,
    'flow_id': 2139992503996592,
    '

In [6]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': '3d97bbc7e7b8',
   'name': 'cve-2021-22205-gitlab-1-proxy',
   'image': 'nginx:alpine',
   'status': 'running',
   'created': '2025-10-14T10:24:36.722830117Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.10'},
  {'id': 'd5e7ad115690',
   'name': 's2-057-struts2-1',
   'image': 'vulhub/struts2:2.3.34-showcase',
   'status': 'running',
   'created': '2025-10-14T10:24:36.345390052Z',
   'ports': ['8080/tcp'],
   'ip_address': '172.20.0.3'},
  {'id': '65c11d7f29b4',
   'name': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'status': 'running',
   'created': '2025-10-14T10:24:36.043252751Z',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip_address': '172.20.0.2'},
  {'id': 'e2e7e2ac93a4',
   'name': 'xdebug-rce-xdebug2-1',
   'image': 'vulhub/php:7.1-xdebug',
   'status': 'running',
   'created': '2025-10-14T10:24:34.66538461Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'}]}

In [8]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [7]:
await firewall_tools.remove_firewall_rule([1, 2])

INFO:tools.firewall_tools:Removing firewall rules: [1, 2]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  172.20.0.8           192.168.100.0/24',
  'REMOVED rule #2:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.8']}

In [179]:
honeypot_ip = '172.20.0.10' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip, port=8080)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.10 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.10 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.10 -> 192.168.100.2/24 (tcp)'}